# Artificial information filtering

In simple terms the bitinformation is retrieved by checking how variable a bit pattern is. However, this approach cannot distinguish between actual information content and artifical information content. By studying the distribution of the information content the user can often identify clear cut-offs of real information content and artificial information content.

The following example shows how such a separation of real information and artificial information can look like. To do so, artificial information is artificially added to an example dataset by applying linear quantization. Linear quantization is often applied to climate datasets (e.g. ERA5) and needs to be accounted for in order to retrieve meaningful bitinformation content. An algorithm that aims at detecting this artificial information itself is introduced.

In [ ]:
import xarray as xr
import xbitinfo as xb
import numpy as np

## Loading example dataset
We use here the openly accessible CONUS dataset. The dataset is available at full precision.

In [ ]:
ds = xr.open_zarr(
    "s3://hytest/conus404/conus404_hourly.zarr",
    storage_options={
        "anon": True,
        "requester_pays": False,
        "client_kwargs": {"endpoint_url": "https://usgs.osn.mghpcc.org"},
    },
)
# selecting water vapor mixing ratio at 2 meters
data = ds["Q2"]
# select subset of data for demonstration purposes
chunk = data.isel(time=slice(0, 9), y=slice(0, 525), x=slice(0, 525))
chunk

## Creating dataset copy with artificial information
### Functions to encode and decode

In [ ]:
# Encoding function to compress data
def encode(chunk, scale, offset, dtype, astype):
    enc = (chunk - offset) * scale
    enc = np.around(enc)
    enc = enc.astype(astype, copy=False)
    return enc


# Decoding function to decompress data
def decode(enc, scale, offset, dtype, astype):
    dec = (enc / scale) + offset
    dec = dec.astype(dtype, copy=False)
    return dec

### Transform dataset to introduce artificial information

In [ ]:
xmin = np.min(chunk)
xmax = np.max(chunk)
scale = (2**16 - 1) / (xmax - xmin)
offset = xmin
enc = encode(chunk, scale, offset, "f4", "u2")
dec = decode(enc, scale, offset, "f4", "u2")

## Comparison of bitinformation

In [ ]:
# original dataset without artificial information
orig_info = xb.get_bitinformation(
    xr.Dataset({"w/o artif. info": chunk}),
    dim="x",
    implementation="python",
    masked_value=None,
)

# dataset with artificial information
arti_info = xb.get_bitinformation(
    xr.Dataset({"w artif. info": dec}),
    dim="x",
    implementation="python",
    masked_value=None,
)

# plotting distribution of bitwise information content
info = xr.merge([orig_info, arti_info])
plot = xb.plot_bitinformation(info)

The figure reveals that artificial information is introduced by applying linear quantization. 

In [ ]:
keepbits = xb.get_keepbits(info, inflevel=[0.99])
print(
    f"The number of keepbits increased from {keepbits['w/o artif. info'].item(0)} bits in the original dataset to {keepbits['w artif. info'].item(0)} bits in the dataset with artificial information."
)

In the following, a gradient based filter is introduced to remove this artificial information again so that even in case artificial information is present in a dataset the number of keepbits remains similar.

## Artificial information filter
The filter `gradient` works as follows:

1. It determines the Cumulative Distribution Function(CDF) of the bitwise information content
2. It computes the gradient of the CDF to identify points where the gradient becomes close to a given tolerance indicating a drop in information.
3. Simultaneously, it keeps track of the minimum cumulative sum of information content which is threshold here, which signifies at least this much fraction of total information needs to be passed.
4. So the bit where the intersection of the gradient reaching the tolerance and the cumulative sum exceeding the threshold is our TrueKeepbits. All bits beyond this index are assumed to contain artificial information and are set to zero in order to cut them off.
5. You can see the above concept implemented in the function get_cdf_without_artificial_information in xbitinfo.py

Please note that this filter relies on a clear separation between real and artificial information content and might not work in all cases.

In [ ]:
xb.get_keepbits(
    arti_info,
    inflevel=[0.99],
    information_filter="Gradient",
    **{"threshold": 0.7, "tolerance": 0.001}
)

With the application of the filter the keepbits are closer/identical to their original value in the dataset without artificial information. The plot of the bitinformation visualizes this:

In [ ]:
plot = xb.plot_bitinformation(arti_info, information_filter="Gradient")